In [1]:
# 批量返回结果解析（OK 与 ERROR JSONL）
import os, re, json
from typing import Optional, List, Any, Dict
import pandas as pd
from ast import literal_eval

# 文件路径
ok_path = os.path.join('.', '中间文件', 'batch_return_files', 'output_202509080020.jsonl')
err_path = os.path.join('.', '中间文件', 'batch_return_files', 'error_202509080020.jsonl')


def extract_code_block_text(text: str) -> str:
    """从三引号代码块中提取内容（若存在），否则返回原文。支持```json/```等。"""
    if not isinstance(text, str):
        return '' if text is None else str(text)
    m = re.search(r"```(?:json)?\s*([\s\S]*?)```", text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    return text.strip()


def parse_responsibilities_from_content(content: Any) -> Optional[List[str]]:
    """将模型 content 转为职责列表。
    兼容形式：
    - 代码块包裹的 JSON：```json {"responsibilities": [...]} ```
    - 纯字符串 JSON：{"responsibilities": [...]} 或 直接返回列表
    - 无法解析则返回 None
    """
    if content is None:
        return None
    if isinstance(content, list):
        return [str(x).strip() for x in content]

    text = extract_code_block_text(str(content))

    # 优先尝试 json.loads
    try:
        obj = json.loads(text)
        if isinstance(obj, dict) and isinstance(obj.get('responsibilities'), list):
            return [str(x).strip() for x in obj['responsibilities']]
        if isinstance(obj, list):
            return [str(x).strip() for x in obj]
    except Exception:
        pass

    # 再尝试 literal_eval（兼容单引号等）
    try:
        obj = literal_eval(text)
        if isinstance(obj, dict) and isinstance(obj.get('responsibilities'), list):
            return [str(x).strip() for x in obj['responsibilities']]
        if isinstance(obj, list):
            return [str(x).strip() for x in obj]
    except Exception:
        pass

    # 简单兜底：若出现明确的 none/null 表达
    if text.strip().lower() in {'none', 'null', '无', '没有'}:
        return None

    return None


In [2]:
# 解析 OK JSONL -> batch_ok_df
ok_rows = []
with open(ok_path, 'r', encoding='utf-8') as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        custom_id = obj.get('custom_id')
        resp = obj.get('response', {})
        body = resp.get('body', {}) or {}
        choices = (body.get('choices') or [])
        content = None
        if choices:
            content = ((choices[0] or {}).get('message') or {}).get('content')
        usage = body.get('usage', {}) or {}
        model = body.get('model')
        status_code = resp.get('status_code')
        responsibilities = parse_responsibilities_from_content(content)
        ok_rows.append({
            'custom_id': custom_id,
            'status_code': status_code,
            'model': model,
            'prompt_tokens': usage.get('prompt_tokens'),
            'completion_tokens': usage.get('completion_tokens'),
            'total_tokens': usage.get('total_tokens'),
            'raw_content': content,
            'responsibilities': responsibilities
        })

batch_ok_df = pd.DataFrame(ok_rows)
for col in ['prompt_tokens', 'completion_tokens', 'total_tokens']:
    batch_ok_df[col] = pd.to_numeric(batch_ok_df[col], errors='coerce').astype('Int64')

len(batch_ok_df), batch_ok_df.head(3)


<unknown>:1: SyntaxWarning: invalid escape sequence '\D'
<unknown>:1: SyntaxWarning: invalid escape sequence '\C'
<unknown>:1: SyntaxWarning: invalid escape sequence '\W'
<unknown>:1: SyntaxWarning: invalid escape sequence '\W'


(49960,
      custom_id  status_code        model  prompt_tokens  completion_tokens  \
 0  resp-102424          200  glm-4-flash            106                 29   
 1  resp-102617          200  glm-4-flash            200                 44   
 2  resp-102684          200  glm-4-flash            202                 59   
 
    total_tokens                                        raw_content  \
 0           135  ```json\n{"responsibilities":["投标标书（含报价文件）制作",...   
 1           244  {"responsibilities":["基于电脑的自动化设备软件开发", "独立开发自动...   
 2           261  {"responsibilities":["从事Android智能手机界面，应用软件开发；"...   
 
                                     responsibilities  
 0                        [投标标书（含报价文件）制作, 合同商务, 回款管理]  
 1  [基于电脑的自动化设备软件开发, 独立开发自动化设备控制软件, 独立完成客户需求收集, 设计...  
 2  [从事Android智能手机界面，应用软件开发；, 根据产品功能需求设计并完成软件实现；, ...  )

In [3]:
# 解析 ERROR JSONL -> batch_err_df
err_rows = []
with open(err_path, 'r', encoding='utf-8') as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        custom_id = obj.get('custom_id')
        resp = obj.get('response', {})
        status_code = resp.get('status_code')
        body = resp.get('body', {}) or {}
        model = body.get('model') or (resp.get('body') or {}).get('model')
        error_obj = obj.get('error') or body.get('error') or {}
        error_code = (error_obj or {}).get('code')
        error_message = (error_obj or {}).get('message')
        err_rows.append({
            'custom_id': custom_id,
            'status_code': status_code,
            'model': model,
            'error_code': error_code,
            'error_message': error_message
        })

batch_err_df = pd.DataFrame(err_rows)
len(batch_err_df), batch_err_df.head(3)


(40,
      custom_id  status_code        model error_code    error_message
 0  resp-102456          400  glm-4-flash       1213  未正常接收到prompt参数。
 1  resp-109548          400  glm-4-flash       1213  未正常接收到prompt参数。
 2  resp-108742          400  glm-4-flash       1213  未正常接收到prompt参数。)

In [4]:
# 预览统计与样例
print('OK数:', len(batch_ok_df))
print('ERROR数:', len(batch_err_df))

# OK 的 responsibilities 非空占比
non_empty = batch_ok_df['responsibilities'].map(lambda x: isinstance(x, list) and len(x) > 0)
print('OK中解析出职责的条数:', non_empty.sum())

batch_ok_df.head(5), batch_err_df.head(5)


OK数: 49960
ERROR数: 40
OK中解析出职责的条数: 44544


(     custom_id  status_code        model  prompt_tokens  completion_tokens  \
 0  resp-102424          200  glm-4-flash            106                 29   
 1  resp-102617          200  glm-4-flash            200                 44   
 2  resp-102684          200  glm-4-flash            202                 59   
 3  resp-102731          200  glm-4-flash            201                 50   
 4  resp-102740          200  glm-4-flash            105                 58   
 
    total_tokens                                        raw_content  \
 0           135  ```json\n{"responsibilities":["投标标书（含报价文件）制作",...   
 1           244  {"responsibilities":["基于电脑的自动化设备软件开发", "独立开发自动...   
 2           261  {"responsibilities":["从事Android智能手机界面，应用软件开发；"...   
 3           251  {"responsibilities":["在上级的领导和监督下定期完成量化的工作要求，并能...   
 4           163  {"responsibilities":["协助市场专员收集市场信息及行业动态","协助市场...   
 
                                     responsibilities  
 0                        [投标标书（含报价文件）制作

In [7]:
# 解析基础请求 JSONL -> batch_req_df
req_rows = []
with open(os.path.join('.', '中间文件', 'zhipu_batches', 'batch_responsibilities_zhipu_003.jsonl'), 'r', encoding='utf-8') as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        custom_id = obj.get('custom_id')
        body = obj.get('body', {}) or {}
        messages = body.get('messages') or []
        model = body.get('model')
        temperature = body.get('temperature')
        max_tokens = body.get('max_tokens')
        # 抓取 system/user 文本，便于回查
        sys_text = None
        user_text = None
        for m in messages:
            role = m.get('role')
            if role == 'system' and sys_text is None:
                sys_text = m.get('content')
            if role == 'user' and user_text is None:
                user_text = m.get('content')
        req_rows.append({
            'custom_id': custom_id,
            'req_model': model,
            'req_temperature': temperature,
            'req_max_tokens': max_tokens,
            'req_system': sys_text,
            'req_user': user_text
        })

batch_req_df = pd.DataFrame(req_rows)
len(batch_req_df), batch_req_df.head(3)


(50000,
      custom_id    req_model  req_temperature req_max_tokens  \
 0  resp-100000  glm-4-flash              0.1           None   
 1  resp-100001  glm-4-flash              0.1           None   
 2  resp-100002  glm-4-flash              0.1           None   
 
                                           req_system  \
 0  请提取岗位描述信息中的岗位职责或任务，输出{"responsibilities":["…",...   
 1  请提取岗位描述信息中的岗位职责或任务，输出{"responsibilities":["…",...   
 2  请提取岗位描述信息中的岗位职责或任务，输出{"responsibilities":["…",...   
 
                                             req_user  
 0  岗位职责:1.食品原料、天然提取物中含量、杂质与残留的分析化验；2.HPLC、GC或GC-M...  
 1  职位描述：1.严格遵守公司规章制度，遵守公司劳动纪律。2.服从电分领班的工作安排。3.负责客...  
 2  一、任职要求：1、大专以上学历，年龄30-45岁，管理类、市场营销类或国际贸易等相关专业。2...  )

In [8]:
# 按 custom_id 合并：请求 + OK + ERROR
# 有的 custom_id 只会出现在 OK 或 ERROR 之一
merged_all = (batch_req_df
              .merge(batch_ok_df, on='custom_id', how='left', suffixes=('', '_ok'))
              .merge(batch_err_df, on='custom_id', how='left', suffixes=('', '_err')))

print('总请求数:', len(batch_req_df))
print('匹配到OK的数:', merged_all['status_code'].notna().sum())
print('匹配到ERROR的数:', merged_all['error_code'].notna().sum())

merged_all.head(5)


总请求数: 50000
匹配到OK的数: 49960
匹配到ERROR的数: 40


,custom_id,req_model,req_temperature,req_max_tokens,req_system,req_user,status_code,model,prompt_tokens,completion_tokens,total_tokens,raw_content,responsibilities,status_code_err,model_err,error_code,error_message
0,resp-100000,glm-4-flash,0.1,None,"请提取岗位描述信息中的岗位职责或任务，输出{""responsibilities"":[""…"",...",岗位职责:1.食品原料、天然提取物中含量、杂质与残留的分析化验；2.HPLC、GC或GC-M...,200.0,glm-4-flash,146,51,197,"{""responsibilities"":[""食品原料、天然提取物中含量、杂质与残留的分析化验...","[食品原料、天然提取物中含量、杂质与残留的分析化验, HPLC、GC或GC-MS等分析仪器的...",NaN,NaN,NaN,NaN
1,resp-100001,glm-4-flash,0.1,None,"请提取岗位描述信息中的岗位职责或任务，输出{""responsibilities"":[""…"",...",职位描述：1.严格遵守公司规章制度，遵守公司劳动纪律。2.服从电分领班的工作安排。3.负责客...,200.0,glm-4-flash,223,101,324,"```json\n{""responsibilities"":[""严格遵守公司规章制度，遵守公司...","[严格遵守公司规章制度，遵守公司劳动纪律, 服从电分领班的工作安排, 负责客来原稿的扫描工作...",NaN,NaN,NaN,NaN
2,resp-100002,glm-4-flash,0.1,None,"请提取岗位描述信息中的岗位职责或任务，输出{""responsibilities"":[""…"",...",一、任职要求：1、大专以上学历，年龄30-45岁，管理类、市场营销类或国际贸易等相关专业。2...,200.0,glm-4-flash,214,51,265,"{""responsibilities"":[""负责区域行业信息调研和整合项目分析"",""负责区域...","[负责区域行业信息调研和整合项目分析, 负责区域整合项目客户开发、项目拓展、项目导入与资源整...",NaN,NaN,NaN,NaN
3,resp-100003,glm-4-flash,0.1,None,"请提取岗位描述信息中的岗位职责或任务，输出{""responsibilities"":[""…"",...",职位描述: 工作内容： 1、国际卖场年度合作协议的谈判或参与： 2、公司销售政策执行； 3、...,200.0,glm-4-flash,189,54,243,"{""responsibilities"":[""国际卖场年度合作协议的谈判或参与"", ""公司销售...","[国际卖场年度合作协议的谈判或参与, 公司销售政策执行, 公司新品推广及促销活动组织和执行,...",NaN,NaN,NaN,NaN
4,resp-100004,glm-4-flash,0.1,None,"请提取岗位描述信息中的岗位职责或任务，输出{""responsibilities"":[""…"",...",职位要求：性别：男学历：全日制本科及以上专业：物资管理、物流仓储管理等相关专业户籍：不限经验...,200.0,glm-4-flash,372,144,516,"{""responsibilities"":[""负责仓储业务及运输业务的运营及调整"", ""负责货...","[负责仓储业务及运输业务的运营及调整, 负责货物的出入库管理，合理存放，并保护好货物，做好标...",NaN,NaN,NaN,NaN


In [16]:
merged_all[merged_all['status_code'].isna()].sample(10).to_excel('preview.xlsx')

In [13]:
merged_all.shape

(50000, 17)